In [ ]:
import numpy as np
import h5py
import math
import os
import pathlib
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Activation, Layer, ReLU, LeakyReLU, Dropout
from tensorflow.keras import backend as K

In [ ]:
from func import load_model, save_model

https://mpp-hep.github.io/ADC2021/

```
python3 create_datasets.py --bkg_file background_for_training.h5 --output_bkg_name bkg --signals_files Ato4l_lepFilter_13TeV.h5 --output_signal_names signal_1 --signals_files hChToTauNu_13TeV_PU20.h5 --output_signal_names signal_2 --signals_files hToTauTau_13TeV_PU20.h5 --output_signal_names signal_3 --signals_files leptoquark_LOWMASS_lepFilter_13TeV.h5 --output_signal_names signal_4
```

## Load dataset

In [ ]:
filename = 'bkg_dataset.h5'

In [ ]:
# make sure input data has correct input shape - background training data
with h5py.File(filename, 'r') as file:
    X_train = np.array(file['X_train'])
    X_test = np.array(file['X_test'])
    X_val = np.array(file['X_val'])

## Define Dense NN architecture

In [ ]:
input_shape = 57
latent_dimension = 128
num_nodes=[32,32]

In [ ]:
#encoder
inputArray = Input(shape=(input_shape))
x = Dense(num_nodes[0], use_bias=False)(inputArray)
x = Activation('relu')(x)
x = Dense(latent_dimension, use_bias=False)(x)
encoder = Activation('relu')(x)

#decoder
x = Dense(num_nodes[0], use_bias=False)(encoder)
x = Activation('relu')(x)
decoder = Dense(input_shape)(x)

#create autoencoder
autoencoder = Model(inputs = inputArray, outputs=decoder)
autoencoder.summary()

In [ ]:
autoencoder.compile(optimizer = keras.optimizers.Adam(lr=1e-2), loss='mse')

## Train model

In [ ]:
EPOCHS = 100
BATCH_SIZE = 5*1024

In [ ]:
history = autoencoder.fit(X_train, X_train, epochs = EPOCHS, batch_size = BATCH_SIZE,
                  validation_data=(X_val, X_val))

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.ylim(0,0.1)

In [ ]:
model_name = 'dense'
model_directory = 'models/'
save_model(model_directory+model_name, autoencoder)

## Prediction - background

In [ ]:
bkg_prediction = autoencoder.predict(X_test)

## Prediction - signals

In [ ]:
# add correct signal labels
signal_labels = ['signal_1','signal_2', 'signal_3', 'signal_4']

In [ ]:
# add correct path to signal files
signals_file = ['signal_1_dataset.h5', 'signal_2_dataset.h5', 'signal_3_dataset.h5', 'signal_4_dataset.h5']

In [ ]:
# read signal data
signal_data = []
for i, label in enumerate(signal_labels):
    with h5py.File(signals_file[i], 'r') as file:
        test_data = np.array(file['Data'])
    signal_data.append(test_data)

In [ ]:
signal_results = []

for i, label in enumerate(signal_labels):
    signal_prediction = autoencoder.predict(signal_data[i])
    signal_results.append([label, signal_data[i], signal_prediction]) # save [label, true, prediction] for signal

## Save results

In [ ]:
save_file = 'predictions.h5'

In [ ]:
with h5py.File(save_file, 'w') as file:
    file.create_dataset('BKG_input', data=X_test)
    file.create_dataset('BKG_predicted', data = bkg_prediction)
    for i, sig in enumerate(signal_results):
        file.create_dataset('%s_input' %sig[0], data=sig[1])
        file.create_dataset('%s_predicted' %sig[0], data=sig[2])

## Evaluate results

1. Plot loss distribution after prediction (check loss value for signals)
2. Plot ROC curves - how good is anomaly detection for chosen FPR threshold

# 1.

In [ ]:
from func import mse_loss

In [ ]:
# compute loss value (true, predicted)
total_loss = []
loss_labels = []

total_loss.append(mse_loss(X_test, bkg_prediction.astype(np.float32)).numpy())
loss_labels.append("bkg")

for i, signal_X in enumerate(signal_data):
    total_loss.append(mse_loss(signal_X, signal_results[i][2].astype(np.float32)).numpy())
    loss_labels.append(signal_labels[i])

In [ ]:
bin_size=100

plt.figure(figsize=(10,8))
for loss, label in zip(total_loss, loss_labels):
    plt.hist(loss, bins=np.logspace(-1,5,100), label=label, density = True, histtype='step', fill=False, linewidth=1.5)
plt.yscale('log')
plt.xscale("log")
plt.xlabel("Autoencoder Loss")
plt.ylabel("Probability (a.u.)")
plt.title('MSE loss')
plt.legend(loc='best')
plt.show()

# 2.

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
labels = np.concatenate([['Background'], np.array(signal_labels)])

In [ ]:
target_background = np.zeros(total_loss[0].shape[0])

plt.figure(figsize=(10,8))
for i, label in enumerate(labels):
    if i == 0: continue # background events
    
    trueVal = np.concatenate((np.ones(total_loss[i].shape[0]), target_background)) # anomaly=1, bkg=0
    predVal_loss = np.concatenate((total_loss[i], total_loss[0]))

    fpr_loss, tpr_loss, threshold_loss = roc_curve(trueVal, predVal_loss)

    auc_loss = auc(fpr_loss, tpr_loss)
    
    plt.plot(fpr_loss, tpr_loss, "-", label='%s (auc = %.1f%%)'%(label,auc_loss*100.), linewidth=1.5)
    
    plt.semilogx()
    plt.semilogy()
    plt.ylabel("True Positive Rate")
    plt.xlabel("False Positive Rate")
    plt.legend(loc='center right')
    plt.grid(True)
    plt.tight_layout()
plt.plot(np.linspace(0, 1),np.linspace(0, 1), '--', color='0.75')
plt.axvline(0.00001, color='red', linestyle='dashed', linewidth=1) # threshold value for measuring anomaly detection efficiency
plt.title("ROC AE")
plt.show()